# Titanic LGBM Prediction

## 탐색적 데이터 분석 (EDA) 및 시각화

In [7]:
print('a")

SyntaxError: unterminated string literal (detected at line 1) (2529827067.py, line 1)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# NOTE: 향후 분석 진행 시, 실제로 사용되지 않는 라이브러리는 제거하여 메모리 사용량을 미세하게 줄일 수 있습니다.

In [ ]:
# Set global plot style for better aesthetics and readability
sns.set_style("whitegrid")
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 100
plt.rcParams['font.size'] = 10 # Base font size
plt.rcParams['axes.titlesize'] = 14 # Title font size
plt.rcParams['axes.labelsize'] = 12 # Label font size
plt.rcParams['xtick.labelsize'] = 10 # X-tick label size
plt.rcParams['ytick.labelsize'] = 10 # Y-tick label size
plt.rcParams['legend.fontsize'] = 10 # Legend font size

In [ ]:
# Load the dataset
# Assuming 'train.csv' is in the same directory or specified path
train_df = pd.read_csv('train.csv')

In [ ]:
# --- Helper Functions for Exploratory Data Analysis (EDA) ---

In [ ]:
def display_basic_info(df):
    """
    Prints basic information about the DataFrame, including its structure,
    first few rows, and statistical summary.
    """
    print("\n--- 1. 데이터 기본 정보 확인 ---")
    print("\n데이터셋 정보:")
    df.info()
    print("\n데이터셋의 처음 5행:")
    print(df.head())
    print("\n데이터셋의 통계적 요약:")
    print(df.describe())

In [ ]:
def analyze_missing_values(df):
    """
    Identifies and visualizes missing values across DataFrame columns.
    """
    print("\n--- 2. 결측치 확인 및 시각화 ---")
    missing_data = df.isnull().sum()
    missing_percentage = (missing_data / len(df)) * 100
    missing_info = pd.DataFrame({
        'Missing Count': missing_data,
        'Missing Percentage (%)': missing_percentage
    })
    missing_info = missing_info[missing_info['Missing Count'] > 0].sort_values(by='Missing Count', ascending=False)

    if not missing_info.empty:
        print("\n결측치 정보:")
        print(missing_info)
        plt.figure(figsize=(10, 6))
        sns.barplot(x=missing_info.index, y='Missing Percentage (%)', data=missing_info, palette='viridis')
        plt.title('각 컬럼별 결측치 비율')
        plt.ylabel('결측치 비율 (%)')
        plt.xlabel('컬럼명')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.show()
    else:
        print("결측치가 없습니다.")

In [6]:
def analyze_target_variable(df, target_col):
    """
    Analyzes and visualizes the distribution of the target variable.
    """
    print(f"\n--- 3. 타겟 변수 ({target_col}) 분포 확인 ---")
    plt.figure(figsize=(6, 4))
    sns.countplot(x=target_col, data=df, palette='pastel')
    plt.title(f'{target_col} 분포')
    plt.xlabel('생존 여부 (0: 사망, 1: 생존)')
    plt.ylabel('승객 수')
    plt.xticks(ticks=[0, 1], labels=['사망', '생존'])
    plt.tight_layout()
    plt.show()

    survival_rate = df[target_col].value_counts(normalize=True) * 100
    if 0 in survival_rate.index and 1 in survival_rate.index:
        print(f"사망자 비율: {survival_rate[0]:.2f}%")
        print(f"생존자 비율: {survival_rate[1]:.2f}%")
    elif 0 in survival_rate.index:
        print(f"사망자 비율: {survival_rate[0]:.2f}%")
        print("생존자 없음.")
    elif 1 in survival_rate.index:
        print("사망자 없음.")
        print(f"생존자 비율: {survival_rate[1]:.2f}%")
    else:
        print("타겟 변수에 0 또는 1 값이 없습니다.")

In [ ]:
def univariate_analysis(df, categorical_features, numerical_features):
    """
    Performs univariate analysis for categorical (count plots) and
    numerical (histograms and box plots) features.
    """
    print("\n--- 4. 각 특징들의 분포 시각화 (유니바리에이트 분석) ---")

    if categorical_features:
        print("\n4.1. 범주형 특징 분포:")
        n_cat = len(categorical_features)
        n_cols_cat = 3
        n_rows_cat = (n_cat + n_cols_cat - 1) // n_cols_cat # Ceiling division
        fig_cat, axes_cat = plt.subplots(n_rows_cat, n_cols_cat, figsize=(n_cols_cat * 6, n_rows_cat * 5))
        axes_cat = axes_cat.flatten() # Ensure axes is a 1D array even for single row/column

        for i, feature in enumerate(categorical_features):
            if i < len(axes_cat):
                sns.countplot(x=feature, data=df, ax=axes_cat[i], palette='coolwarm')
                axes_cat[i].set_title(f'{feature} 분포')
                axes_cat[i].set_ylabel('승객 수')
                axes_cat[i].set_xlabel(feature)

        # Remove unused subplots if any
        for j in range(n_cat, len(axes_cat)):
            fig_cat.delaxes(axes_cat[j])

        plt.suptitle('범주형 특징 분포', y=1.02, fontsize=16)
        plt.tight_layout(rect=[0, 0, 1, 0.98]) # Adjust rect to prevent title overlap
        plt.show()
    else:
        print("\n4.1. 분석할 범주형 특징이 없습니다.")

    if numerical_features:
        print("\n4.2. 수치형 특징 분포 (히스토그램 및 박스 플롯):")
        n_num = len(numerical_features)
        fig_num, axes_num = plt.subplots(n_num, 2, figsize=(15, 5 * n_num))
        axes_num = axes_num.flatten()

        for i, feature in enumerate(numerical_features):
            # Histplot
            sns.histplot(df[feature].dropna(), kde=True, ax=axes_num[2*i], bins=30, color='skyblue')
            axes_num[2*i].set_title(f'{feature} 분포 (히스토그램)')
            axes_num[2*i].set_xlabel(feature)
            axes_num[2*i].set_ylabel('밀도 / 승객 수')

            # Boxplot
            sns.boxplot(y=df[feature].dropna(), ax=axes_num[2*i+1], color='lightcoral')
            axes_num[2*i+1].set_title(f'{feature} 분포 (박스 플롯)')
            axes_num[2*i+1].set_ylabel(feature)

        plt.suptitle('수치형 특징 분포', y=1.02, fontsize=16)
        plt.tight_layout(rect=[0, 0, 1, 0.98])
        plt.show()
    else:
        print("\n4.2. 분석할 수치형 특징이 없습니다.")

In [ ]:
def bivariate_analysis(df, target_col, categorical_features, numerical_features):
    """
    Analyzes relationships between features and the target variable ('Survived').
    """
    print(f"\n--- 5. {target_col}와 다른 특징들 간의 관계 분석 (바이바리에이트 분석) ---")

    if categorical_features:
        print(f"\n5.1. 범주형 특징 vs. {target_col}:")
        n_cat = len(categorical_features)
        n_cols_cat = 3
        n_rows_cat = (n_cat + n_cols_cat - 1) // n_cols_cat
        fig_cat, axes_cat = plt.subplots(n_rows_cat, n_cols_cat, figsize=(n_cols_cat * 6, n_rows_cat * 5))
        axes_cat = axes_cat.flatten()

        for i, feature in enumerate(categorical_features):
            if i < len(axes_cat):
                sns.barplot(x=feature, y=target_col, data=df, ax=axes_cat[i], palette='viridis', errorbar=None)
                axes_cat[i].set_title(f'{feature}별 {target_col}율')
                axes_cat[i].set_ylabel(f'{target_col}율')
                axes_cat[i].set_xlabel(feature)

        for j in range(n_cat, len(axes_cat)):
            fig_cat.delaxes(axes_cat[j])

        plt.suptitle(f'범주형 특징별 {target_col}율', y=1.02, fontsize=16)
        plt.tight_layout(rect=[0, 0, 1, 0.98])
        plt.show()
    else:
        print(f"\n5.1. 분석할 범주형 특징이 없습니다.")

    if numerical_features:
        print(f"\n5.2. 수치형 특징과 {target_col} 관계:")
        n_num = len(numerical_features)
        fig_num, axes_num = plt.subplots(n_num, 2, figsize=(15, 6 * n_num))
        axes_num = axes_num.flatten()

        for i, feature in enumerate(numerical_features):
            # KDE plot
            sns.kdeplot(x=feature, data=df, hue=target_col, fill=True, common_norm=False, ax=axes_num[2*i], palette='mako')
            axes_num[2*i].set_title(f'{feature} 분포 vs. {target_col}')
            axes_num[2*i].set_xlabel(feature)
            axes_num[2*i].set_ylabel('밀도')
            # Custom legend labels for Survived (0: 사망, 1: 생존)
            handles, labels = axes_num[2*i].get_legend_handles_labels()
            if '0' in labels and '1' in labels: # Check if default labels exist before modifying
                labels_map = {'0': '사망', '1': '생존'}
                new_labels = [labels_map.get(l, l) for l in labels]
                axes_num[2*i].legend(handles=handles, labels=new_labels, title=target_col)
            else:
                axes_num[2*i].legend(title=target_col)


            # Box plot
            sns.boxplot(x=target_col, y=feature, data=df, ax=axes_num[2*i+1], palette='rocket')
            axes_num[2*i+1].set_title(f'{target_col}별 {feature} 분포')
            axes_num[2*i+1].set_xlabel('생존 여부 (0: 사망, 1: 생존)')
            axes_num[2*i+1].set_ylabel(feature)
            axes_num[2*i+1].set_xticks(ticks=[0, 1], labels=['사망', '생존'])

        plt.suptitle(f'수치형 특징별 {target_col} 관계', y=1.02, fontsize=16)
        plt.tight_layout(rect=[0, 0, 1, 0.98])
        plt.show()
    else:
        print(f"\n5.2. 분석할 수치형 특징이 없습니다.")

In [ ]:
def analyze_correlations(df, target_col=None):
    """
    Calculates and visualizes the correlation matrix for numerical features.
    """
    print("\n--- 6. 수치형 특징 간의 상관 관계 분석 ---")

    numeric_cols_for_corr = df.select_dtypes(include=np.number).columns.tolist()
    if 'PassengerId' in numeric_cols_for_corr:
        numeric_cols_for_corr.remove('PassengerId')

    if not numeric_cols_for_corr:
        print("상관 관계를 분석할 수치형 특징이 없습니다.")
        return

    correlation_matrix = df[numeric_cols_for_corr].corr()

    plt.figure(figsize=(10, 8))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5, cbar_kws={'label': '상관 계수'})
    plt.title('수치형 특징 간의 상관 관계 매트릭스')
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()

In [ ]:

# --- Main EDA Execution Flow ---
if __name__ == "__main__":
    print("--- 탐색적 데이터 분석 (EDA) 시작 ---")

    # Define the target column and features to be excluded from general analysis
    TARGET_COLUMN = 'Survived'
    EXCLUDE_FEATURES = ['PassengerId', 'Name', 'Ticket', 'Cabin']

    # Explicitly define categorical and numerical features for this dataset
    # These lists can be made dynamic based on data types and unique value counts if needed
    categorical_features = ['Sex', 'Pclass', 'Embarked', 'SibSp', 'Parch']
    numerical_features = ['Age', 'Fare']

    # Filter features to ensure they exist in the DataFrame and are not target/excluded
    available_categorical_features = [f for f in categorical_features if f in train_df.columns and f not in EXCLUDE_FEATURES and f != TARGET_COLUMN]
    available_numerical_features = [f for f in numerical_features if f in train_df.columns and f not in EXCLUDE_FEATURES and f != TARGET_COLUMN]

    # Check if the target column exists before proceeding with EDA
    if TARGET_COLUMN not in train_df.columns:
        print(f"오류: 타겟 컬럼 '{TARGET_COLUMN}'이 데이터프레임에 존재하지 않습니다. EDA를 수행할 수 없습니다.")
    else:
        display_basic_info(train_df)
        analyze_missing_values(train_df)
        analyze_target_variable(train_df, TARGET_COLUMN)
        univariate_analysis(train_df, available_categorical_features, available_numerical_features)
        bivariate_analysis(train_df, TARGET_COLUMN, available_categorical_features, available_numerical_features)
        analyze_correlations(train_df) # analyze_correlations handles numeric column selection internally

    print("\n--- 탐색적 데이터 분석 (EDA) 완료 ---")

## 특징 공학 (Feature Engineering)

In [ ]:
import pandas as pd
import re

In [ ]:
def perform_feature_engineering(train_df_raw: pd.DataFrame, test_df_raw: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    Performs feature engineering on Titanic train and test dataframes.

    Args:
        train_df_raw (pd.DataFrame): The raw training DataFrame, expected to contain
                                     'PassengerId', 'Survived', 'Name', 'SibSp', 'Parch',
                                     'Fare', 'Ticket', 'Cabin', 'Age', 'Embarked', 'Pclass', 'Sex'.
                                     Basic missing value imputation (e.g., Age, Embarked)
                                     is assumed to have been completed.
        test_df_raw (pd.DataFrame): The raw test DataFrame, expected to contain
                                    'PassengerId', 'Name', 'SibSp', 'Parch', 'Fare',
                                    'Ticket', 'Cabin', 'Age', 'Embarked', 'Pclass', 'Sex'.
                                    Basic missing value imputation (e.g., Age)
                                    is assumed to have been completed.

    Returns:
        tuple[pd.DataFrame, pd.DataFrame]: A tuple containing two DataFrames:
                                           - The engineered training DataFrame.
                                           - The engineered test DataFrame.
    """
    # Create copies to avoid modifying the original dataframes passed to the function
    train_df = train_df_raw.copy()
    test_df = test_df_raw.copy()

    # Store 'PassengerId' and 'Survived' target as they are not features for engineering
    train_passenger_ids = train_df['PassengerId']
    test_passenger_ids = test_df['PassengerId']
    train_target = train_df['Survived']

    # Combine dataframes for consistent feature engineering
    # Drop 'Survived' (only in train) and 'PassengerId' from both before combining
    combined_df = pd.concat([train_df.drop(columns=['Survived', 'PassengerId']),
                             test_df.drop(columns=['PassengerId'])],
                            axis=0, ignore_index=True)

    # Store the original length of the training data for later splitting
    train_data_len = len(train_df)

    # --- Feature Engineering Steps ---

    # 1. Extract 'Title' from 'Name'
    # Use str.extract with regex to get the title (e.g., Mr, Mrs, Miss)
    combined_df['Title'] = combined_df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
    
    # Fill any NaNs (if a name doesn't match the regex pattern) with 'Unknown'
    combined_df['Title'] = combined_df['Title'].fillna('Unknown')

    # Standardize and group less common titles
    # Use .loc for explicit assignment to avoid potential SettingWithCopyWarning
    combined_df.loc[combined_df['Title'].isin(['Mlle', 'Ms']), 'Title'] = 'Miss'
    combined_df.loc[combined_df['Title'] == 'Mme', 'Title'] = 'Mrs'
    
    # Define rare titles including the 'Unknown' placeholder and group them
    rare_titles = ['Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev',
                   'Sir', 'Jonkheer', 'Dona', 'Unknown']
    combined_df.loc[combined_df['Title'].isin(rare_titles), 'Title'] = 'Rare'

    # 2. Create 'FamilySize' from 'SibSp' and 'Parch'
    # Family size includes the person themselves, so add 1
    combined_df['FamilySize'] = combined_df['SibSp'] + combined_df['Parch'] + 1

    # 3. Create 'IsAlone' feature
    # 'IsAlone' is 1 if FamilySize is 1, else 0
    combined_df['IsAlone'] = (combined_df['FamilySize'] == 1).astype(int)

    # 4. Calculate 'FarePerPerson'
    # Fill any missing 'Fare' values with the median of the combined dataset
    combined_df['Fare'].fillna(combined_df['Fare'].median(), inplace=True)
    # Calculate fare per person, FamilySize is guaranteed to be >= 1
    combined_df['FarePerPerson'] = combined_df['Fare'] / combined_df['FamilySize']

    # --- Drop original columns that are no longer needed or replaced ---
    # 'Name' is replaced by 'Title'
    # 'SibSp', 'Parch' are replaced by 'FamilySize' and 'IsAlone'
    # 'Ticket', 'Cabin' are dropped for simplicity as per original code's intent
    columns_to_drop = ['Name', 'SibSp', 'Parch', 'Ticket', 'Cabin']
    combined_df = combined_df.drop(columns=columns_to_drop)

    # --- Split the combined dataframe back into engineered train and test sets ---
    train_df_fe = combined_df.iloc[:train_data_len].copy()
    test_df_fe = combined_df.iloc[train_data_len:].copy()

    # Re-add 'Survived' target and 'PassengerId' to the respective dataframes
    train_df_fe['Survived'] = train_target
    train_df_fe['PassengerId'] = train_passenger_ids
    test_df_fe['PassengerId'] = test_passenger_ids

    return train_df_fe, test_df_fe

## 결측치 처리 (Missing Value Imputation)

In [ ]:
# 결측치 확인 (재확인)
print("--- 결측치 처리 전 ---")
print("Train 데이터셋 결측치:\n", train_df.isnull().sum())
print("\nTest 데이터셋 결측치:\n", test_df.isnull().sum())

In [ ]:
# 결측치 처리 값을 train 데이터셋에서 계산하여 데이터 누수 방지
# 1. 'Age' 결측치 처리: 연속형 변수이므로 중앙값(median)으로 대체
median_age = train_df['Age'].median()

In [ ]:
# 2. 'Fare' 결측치 처리: 연속형 변수이므로 중앙값(median)으로 대체
median_fare = train_df['Fare'].median()

In [ ]:
# 3. 'Embarked' 결측치 처리: 범주형 변수이므로 최빈값(mode)으로 대체
most_frequent_embarked = train_df['Embarked'].mode()[0]

In [ ]:
# 각 컬럼별 결측치 대체 값 딕셔너리 정의
imputation_values = {
    'Age': median_age,
    'Fare': median_fare,
    'Embarked': most_frequent_embarked
}

In [ ]:
# train_df와 test_df에 일괄적으로 결측치 처리 적용
for col, value in imputation_values.items():
    train_df[col].fillna(value, inplace=True)
    test_df[col].fillna(value, inplace=True)

In [ ]:
# 결측치 처리 후 확인
print("\n--- 결측치 처리 후 ---")
print("Train 데이터셋 결측치:\n", train_df.isnull().sum())
print("\nTest 데이터셋 결측치:\n", test_df.isnull().sum())

## 범주형 특징 인코딩 (Categorical Feature Encoding)

In [ ]:
import pandas as pd

In [ ]:
# Define the list of categorical columns to be encoded.
# 'Pclass' is treated as categorical.
categorical_features = ['Sex', 'Embarked', 'Title', 'Pclass']

In [ ]:
# Store PassengerId for the test set; it's needed for submission but not for training.
test_passenger_ids = test_df['PassengerId'].copy()

In [ ]:
# Separate the target variable ('Survived') from the training features.
y_train = train_df['Survived'].copy()
X_train = train_df.drop('Survived', axis=1).copy()
X_test = test_df.copy()

In [ ]:
# Drop 'PassengerId' from feature sets as it is an identifier, not a predictive feature.
X_train = X_train.drop('PassengerId', axis=1)
X_test = X_test.drop('PassengerId', axis=1)

In [ ]:
# Concatenate X_train and X_test to apply One-Hot Encoding uniformly.
# This ensures consistent columns across both datasets after encoding.
all_data = pd.concat([X_train, X_test], axis=0, ignore_index=True)

In [ ]:
# Apply One-Hot Encoding to the specified categorical features.
# `drop_first=False` is generally preferred for tree-based models like LightGBM.
all_data_encoded = pd.get_dummies(all_data, columns=categorical_features, drop_first=False)

In [ ]:
# Split the combined dataframe back into processed training and testing sets.
X_train_processed = all_data_encoded.iloc[:len(X_train)].copy()
X_test_processed = all_data_encoded.iloc[len(X_train):].copy()

In [ ]:
# Update the global `train_df` and `test_df` variables with the encoded data.
train_df = X_train_processed
test_df = X_test_processed

In [ ]:
# --- Verification and Display ---
print("--- After Categorical Feature Encoding ---")
print("\nTransformed train_df head:")
print(train_df.head())

In [ ]:
print("\nTransformed test_df head:")
print(test_df.head())

In [ ]:
print(f"\nShape of train_df after encoding: {train_df.shape}")
print(f"Shape of test_df after encoding: {test_df.shape}")
print(f"Number of columns in train_df: {len(train_df.columns)}")
print(f"Number of columns in test_df: {len(test_df.columns)}")
print(f"Number of target values (y_train): {len(y_train)}")

In [ ]:
# Verify column consistency between training and testing sets.
if set(train_df.columns) == set(test_df.columns):
    print("\nSUCCESS: Columns of train_df and test_df are identical, ensuring consistent feature sets.")
else:
    print("\nWARNING: Columns of train_df and test_df are NOT identical. Please check the encoding process.")
    print("Train columns not in Test:", set(train_df.columns) - set(test_df.columns))
    print("Test columns not in Train:", set(test_df.columns) - set(train_df.columns))

## 데이터 분할 및 스케일링 (Data Splitting & Scaling)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
def preprocess_and_split_data(train_df_raw, test_df_raw, 
                              target_column='Survived', 
                              columns_to_exclude_from_scaling=None,
                              test_size=0.2, random_state=42):
    """
    Performs feature/target separation, numerical feature scaling (excluding specified columns),
    and data splitting.

    Args:
        train_df_raw (pd.DataFrame): Raw training data. Assumed to be already preprocessed
                                     for categorical features (e.g., one-hot encoded).
        test_df_raw (pd.DataFrame): Raw test data (without target). Assumed to be already
                                    preprocessed for categorical features.
        target_column (str): The name of the target variable column. Defaults to 'Survived'.
        columns_to_exclude_from_scaling (list, optional): A list of column names that are
                                                          numerical but should NOT be scaled.
                                                          Defaults to ['PassengerId'] if None.
        test_size (float): The proportion of the training data to include in the validation split.
                           Defaults to 0.2.
        random_state (int): Controls the shuffling applied to the data before applying the split.
                            Defaults to 42.

    Returns:
        tuple: (X_train, X_val, y_train, y_val, X_test)
               - X_train (pd.DataFrame): Features for training (scaled).
               - X_val (pd.DataFrame): Features for validation (scaled).
               - y_train (pd.Series): Target for training.
               - y_val (pd.Series): Target for validation.
               - X_test (pd.DataFrame): Processed features for the test set (scaled).
    """
    if columns_to_exclude_from_scaling is None:
        columns_to_exclude_from_scaling = ['PassengerId']

    # Create copies to avoid modifying original dataframes and potential SettingWithCopyWarning
    train_df = train_df_raw.copy()
    X_test = test_df_raw.copy() # X_test will be processed from this copy

    # --- 1. 특징(X)과 타겟(y) 분리 ---
    # 'train_df'는 이전 단계에서 전처리된 훈련 데이터라고 가정합니다.
    # 'TARGET'은 타겟 변수의 컬럼 이름이라고 가정합니다.
    if target_column not in train_df.columns:
        raise KeyError(f"Target column '{target_column}' not found in train_df.")

    # 훈련 데이터에서 타겟 변수를 분리하여 X와 y를 생성합니다.
    X = train_df.drop(columns=[target_column])
    y = train_df[target_column]

    # 'test_df'는 이전 단계에서 전처리된 테스트 데이터라고 가정합니다.
    # 테스트 데이터는 타겟 변수가 없으므로 X_test로 바로 사용합니다.
    # X와 X_test의 컬럼 일치를 보장하기 위해 reindex를 사용합니다.
    # 훈련 세트에만 존재하는 범주형 컬럼이 테스트 세트에 없을 경우 0으로 채워 일치시킵니다.
    # 테스트 세트에만 존재하는 컬럼은 제거합니다.
    
    missing_in_test = set(X.columns) - set(X_test.columns)
    extra_in_test = set(X_test.columns) - set(X.columns)

    if missing_in_test or extra_in_test:
        print("Warning: Features in X and X_test do not perfectly match. Aligning X_test to X's columns.")
        if missing_in_test:
            print(f"Columns missing in X_test: {missing_in_test}. Adding with fill_value=0.")
        if extra_in_test:
            print(f"Columns extra in X_test: {extra_in_test}. Dropping these from X_test.")
        X_test = X_test.reindex(columns=X.columns, fill_value=0) # 0 for one-hot encoded features

    print(f"훈련 데이터 X shape: {X.shape}")
    print(f"훈련 데이터 y shape: {y.shape}")
    print(f"테스트 데이터 X_test shape: {X_test.shape}")


    # --- 2. 숫자형 특징 스케일링 ---
    # LightGBM은 트리 기반 모델이므로 스케일링이 필수적이지는 않지만,
    # 다른 모델과의 비교나 특정 상황에서 성능 향상을 가져올 수 있습니다.
    # 여기서는 일반적인 관행에 따라 숫자형 특징에 StandardScaler를 적용합니다.

    # 숫자형 특징 컬럼을 식별합니다. (여기서는 'PassengerId'와 같은 ID 컬럼은 제외)
    # 'Sex', 'Embarked' 등 범주형 컬럼은 이미 원-핫 인코딩이나 레이블 인코딩되었다고 가정합니다.
    # 'Pclass'는 범주형으로 처리되었을 수 있지만, 여기서는 일단 숫자형으로 간주합니다.
    numerical_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

    # 모델에 직접 사용되지 않는 ID 컬럼 등은 스케일링에서 제외할 수 있습니다.
    # 예를 들어, 'PassengerId'가 X에 남아있다면 스케일링 대상에서 제외합니다.
    scaled_numerical_features = [col for col in numerical_features if col not in columns_to_exclude_from_scaling]

    print(f"\n스케일링 대상 숫자형 특징: {scaled_numerical_features}")

    scaler = StandardScaler()

    if scaled_numerical_features: # 스케일링할 특징이 있는 경우에만 적용
        # 훈련 데이터의 숫자형 특징에 스케일링을 적용합니다.
        # fit_transform을 사용하여 스케일러를 훈련하고 데이터를 변환합니다.
        X[scaled_numerical_features] = scaler.fit_transform(X[scaled_numerical_features])

        # 테스트 데이터의 숫자형 특징에 훈련된 스케일러를 사용하여 변환합니다.
        # 테스트 데이터에는 fit을 다시 호출하지 않고 transform만 사용합니다.
        X_test[scaled_numerical_features] = scaler.transform(X_test[scaled_numerical_features])

        print("숫자형 특징 스케일링 완료.")
        print("X (스케일링 후) 샘플:\n", X[scaled_numerical_features].head())
        print("X_test (스케일링 후) 샘플:\n", X_test[scaled_numerical_features].head())
    else:
        print("스케일링할 숫자형 특징이 없습니다.")


    # --- 3. 훈련 데이터셋을 훈련 세트와 검증 세트로 분할 ---
    # 모델 학습 및 성능 검증을 위해 훈련 데이터를 훈련 세트와 검증 세트로 나눕니다.
    # stratify=y를 사용하여 타겟 변수의 클래스 비율이 훈련 세트와 검증 세트에서 동일하게 유지되도록 합니다.
    # random_state를 고정하여 코드 실행 시마다 동일한 분할 결과를 얻도록 합니다.

    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=test_size, random_state=random_state, stratify=y
    )

    print(f"\n데이터 분할 완료:")
    print(f"X_train shape: {X_train.shape}")
    print(f"y_train shape: {y_train.shape}")
    print(f"X_val shape: {X_val.shape}")
    print(f"y_val shape: {y_val.shape}")

    # 분할된 데이터의 클래스 비율 확인 (선택 사항)
    print(f"\ny_train 클래스 비율:\n{y_train.value_counts(normalize=True)}")
    print(f"y_val 클래스 비율:\n{y_val.value_counts(normalize=True)}")

    # 이 시점에서 X_train, y_train은 모델 학습에 사용될 것이고,
    # X_val, y_val은 모델의 성능을 검증하는 데 사용될 것입니다.
    # X_test는 최종 예측 파일 생성에 사용될 것입니다.
    return X_train, X_val, y_train, y_val, X_test

## LightGBM 모델 학습 (LightGBM Model Training)

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.datasets import make_classification # make_classification 임포트 위치 변경
import warnings

In [ ]:
# 경고 메시지 무시 설정 (LightGBM 등에서 발생하는 경고를 숨기기 위함)
warnings.filterwarnings('ignore')

In [ ]:
# ==============================================================================
# 0. 데이터 로드 및 분할 (가상의 데이터, 실제 노트북에서는 이전 섹션에서 로드됨)
#    이 섹션의 실행을 위해 임시로 X_train, y_train을 생성합니다.
#    실제 환경에서는 이전 단계에서 준비된 'X_train'과 'y_train'을 사용합니다.
# ==============================================================================

In [ ]:
# X_train, y_train이 정의되지 않았는지 확인하고 예시 데이터를 생성
if 'X_train' not in globals() and 'X_train' not in locals():
    print("X_train, y_train이 정의되지 않았습니다. 예시 데이터를 생성합니다.")
    # 예시 데이터 생성 (실제 데이터는 이전 섹션에서 로드 및 전처리됨)
    X_train_temp, y_train_temp = make_classification(
        n_samples=1000, n_features=10, n_informative=5, n_redundant=0,
        n_clusters_per_class=1, random_state=42
    )
    X_train = pd.DataFrame(X_train_temp, columns=[f'feature_{i}' for i in range(X_train_temp.shape[1])])
    y_train = pd.Series(y_train_temp)
    print("예시 데이터 (X_train, y_train) 생성이 완료되었습니다.")
else:
    print("X_train, y_train이 이미 정의되어 있습니다. 기존 데이터를 사용합니다.")

In [ ]:

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_train class distribution:\n{y_train.value_counts()}")

In [ ]:
# ==============================================================================
# 1. LightGBM 모델 초기화
#    `LGBMClassifier`를 사용하여 LightGBM 모델을 초기화합니다.
#    `random_state`를 고정하여 결과의 재현성을 확보합니다.
# ==============================================================================

In [ ]:
print("\n--- 1. LightGBM 모델 초기화 ---")
lgbm = lgb.LGBMClassifier(random_state=42)
print("LightGBM Classifier 모델이 초기화되었습니다.")
print(lgbm)

In [ ]:
# ==============================================================================
# 2. 하이퍼파라미터 튜닝 (Grid Search with Cross-Validation)
#    Grid Search를 사용하여 최적의 하이퍼파라미터를 탐색합니다.
#    교차 검증(StratifiedKFold)을 적용하여 모델의 일반화 성능을 평가합니다.
#    (주의: 파라미터 그리드를 너무 크게 설정하면 실행 시간이 매우 길어질 수 있습니다.)
# ==============================================================================

In [ ]:
print("\n--- 2. 하이퍼파라미터 튜닝 (Grid Search with Cross-Validation) ---")

In [ ]:
# 탐색할 하이퍼파라미터 그리드 정의
# 예시를 위해 간략하게 설정합니다. 실제 환경에서는 더 넓은 범위와 세밀한 값들을 탐색할 수 있습니다.
param_grid = {
    'n_estimators': [100, 200],         # 부스팅 라운드 수
    'learning_rate': [0.05, 0.1],       # 학습률
    'num_leaves': [20, 31],             # 하나의 트리가 가질 수 있는 최대 잎 노드 수
    'max_depth': [5, 7],                # 트리의 최대 깊이
    'min_child_samples': [20, 30]       # 리프 노드가 되기 위한 최소한의 데이터 수
}

In [ ]:
# Stratified K-Fold 교차 검증 설정 (클래스 불균형 문제를 고려하여 사용)
# n_splits: 폴드(fold) 수, shuffle: 데이터 섞기, random_state: 재현성
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
# GridSearchCV 객체 초기화
# estimator: 학습할 모델, param_grid: 탐색할 파라미터 그리드
# scoring: 모델 평가 지표 (예: 'accuracy', 'roc_auc', 'f1')
# cv: 교차 검증 전략, n_jobs: 병렬 처리할 코어 수 (-1은 모든 코어 사용)
# verbose: 진행 상황 출력 레벨 (0: 없음, 1: 요약, 2: 상세)
grid_search = GridSearchCV(
    estimator=lgbm,
    param_grid=param_grid,
    scoring='accuracy',  # 분류 문제이므로 정확도(accuracy)를 사용
    cv=skf,
    n_jobs=-1,           # 모든 CPU 코어 사용
    verbose=2            # 상세한 진행 상황 출력
)

In [ ]:
print("GridSearchCV를 시작합니다. (이 과정은 시간이 다소 소요될 수 있습니다.)")
# 훈련 데이터에 GridSearchCV 적용하여 최적의 파라미터 탐색
grid_search.fit(X_train, y_train)

In [ ]:
print("\n--- GridSearchCV 결과 ---")
# 최적의 하이퍼파라미터 출력
print(f"최적의 하이퍼파라미터: {grid_search.best_params_}")
# 최적의 하이퍼파라미터로 얻은 교차 검증 최고 점수 출력
print(f"최고 교차 검증 정확도: {grid_search.best_score_:.4f}")

In [ ]:
# ==============================================================================
# 3. 최적의 모델로 최종 학습
#    Grid Search를 통해 찾은 최적의 하이퍼파라미터로 LightGBM 모델을 다시 초기화하고,
#    전체 훈련 세트(X_train, y_train)를 사용하여 최종 모델을 학습시킵니다.
# ==============================================================================

In [ ]:
print("\n--- 3. 최적의 모델로 최종 학습 ---")

In [ ]:
# GridSearchCV의 best_estimator_ 속성은 최적의 파라미터로 전체 훈련 세트에 재학습된 모델입니다.
# GridSearchCV의 기본 설정 (refit=True)에 따라, 이 모델은 이미 최적의 파라미터로 전체 훈련 데이터에 학습이 완료된 상태입니다.
final_lgbm_model = grid_search.best_estimator_

In [ ]:
print("GridSearchCV를 통해 최적화된 LightGBM 모델이 준비되었습니다.")
print(f"최종 모델 파라미터:\n{final_lgbm_model.get_params()}")

In [ ]:
# ==============================================================================
# 4. (선택 사항) 학습된 모델의 간단한 성능 확인 (훈련 세트에 대한)
#    훈련 세트에 대한 예측 및 분류 리포트를 출력하여 모델의 학습 상태를 확인합니다.
#    이는 과적합 여부를 판단하는 데 도움이 될 수 있습니다.
#    (주: 실제 모델 평가는 검증 세트나 테스트 세트에서 이루어져야 합니다.)
# ==============================================================================

In [ ]:
print("\n--- 4. (선택 사항) 훈련 세트에 대한 모델 성능 확인 ---")
y_train_pred = final_lgbm_model.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)

In [ ]:
print(f"훈련 세트 정확도: {train_accuracy:.4f}")
print("\n훈련 세트 분류 리포트:")
print(classification_report(y_train, y_train_pred))

In [ ]:
print("\nLightGBM 모델 학습 및 튜닝 섹션이 완료되었습니다.")
# 최종 학습된 모델을 다음 섹션에서 사용할 수 있도록 저장 (예: final_lgbm_model)
# globals()['final_lgbm_model'] = final_lgbm_model

## 모델 평가 (Model Evaluation)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, roc_curve
import matplotlib.pyplot as plt
import seaborn as sns
import os
from typing import Any, Tuple, Dict, Optional

In [ ]:
def evaluate_classification_model(
    model: Any,
    X_val: pd.DataFrame,
    y_val: pd.Series,
    model_name: str = "Model",
    target_names: Optional[list] = None,
    save_plots: bool = False,
    plot_dir: str = "plots"
) -> Tuple[Dict[str, float], plt.Figure, plt.Figure]:
    """
    Evaluates a binary classification model and visualizes its performance.

    Args:
        model: Trained scikit-learn compatible classification model (e.g., LightGBM).
               Must have `predict` and `predict_proba` methods.
        X_val (pd.DataFrame): Validation set features.
        y_val (pd.Series): True labels for the validation set.
        model_name (str): Name of the model for plot titles and legends.
        target_names (Optional[list]): List of target class names for confusion matrix labels.
                                       Defaults to ['0', '1'] if None.
        save_plots (bool): If True, saves the plots to `plot_dir`.
        plot_dir (str): Directory to save plots if `save_plots` is True.
                        Defaults to "plots".

    Returns:
        Tuple[Dict[str, float], plt.Figure, plt.Figure]:
            - A dictionary containing calculated performance metrics.
            - Matplotlib Figure object for the Confusion Matrix.
            - Matplotlib Figure object for the ROC Curve.
    """
    print(f"--- {model_name} Model Evaluation ---")
    print("1. Performing predictions on the validation set...")

    # Class predictions (0 or 1)
    y_pred = model.predict(X_val)
    # Predicted probabilities (probability of class 1)
    y_pred_proba = model.predict_proba(X_val)[:, 1]

    print("Predictions completed.\n")

    print("2. Calculating and printing model performance metrics...")

    metrics = {
        "Accuracy": accuracy_score(y_val, y_pred),
        "Precision": precision_score(y_val, y_pred),
        "Recall": recall_score(y_val, y_pred),
        "F1-Score": f1_score(y_val, y_pred),
        "ROC-AUC": roc_auc_score(y_val, y_pred_proba)
    }

    for metric_name, value in metrics.items():
        print(f"{metric_name}: {value:.4f}")
    print("\n")

    print("3. Visualizing Confusion Matrix...")

    cm = confusion_matrix(y_val, y_pred)
    if target_names is None:
        target_names = ['0', '1']

    cm_fig, ax_cm = plt.subplots(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
                xticklabels=[f'Predicted {name}' for name in target_names],
                yticklabels=[f'Actual {name}' for name in target_names],
                ax=ax_cm)
    ax_cm.set_xlabel('Predicted Label')
    ax_cm.set_ylabel('True Label')
    ax_cm.set_title(f'{model_name} Confusion Matrix')

    if save_plots:
        os.makedirs(plot_dir, exist_ok=True)
        cm_fig.savefig(os.path.join(plot_dir, f'{model_name.lower().replace(" ", "_")}_confusion_matrix.png'))
        plt.close(cm_fig)
    else:
        plt.show()

    print("Confusion Matrix visualization completed.\n")

    print("4. Visualizing ROC Curve...")

    fpr, tpr, thresholds = roc_curve(y_val, y_pred_proba)

    roc_fig, ax_roc = plt.subplots(figsize=(8, 6))
    ax_roc.plot(fpr, tpr, label=f'{model_name} (ROC-AUC = {metrics["ROC-AUC"]:.4f})')
    ax_roc.plot([0, 1], [0, 1], 'k--', label='Random Classifier') # Diagonal line
    ax_roc.set_xlabel('False Positive Rate (FPR)')
    ax_roc.set_ylabel('True Positive Rate (TPR)')
    ax_roc.set_title(f'{model_name} ROC Curve')
    ax_roc.legend()
    ax_roc.grid(True)

    if save_plots:
        os.makedirs(plot_dir, exist_ok=True)
        roc_fig.savefig(os.path.join(plot_dir, f'{model_name.lower().replace(" ", "_")}_roc_curve.png'))
        plt.close(roc_fig)
    else:
        plt.show()

    print("ROC Curve visualization completed.\n")
    print(f"{model_name} model evaluation finished.")

    return metrics, cm_fig, roc_fig

## 예측 및 결과 파일 생성 (Prediction & Submission File Generation)

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb

In [ ]:
# --- Assumed Pre-existing Objects ---
# The following objects are assumed to be available from previous steps when calling the function:
# - lgbm_model: A trained LightGBM model object (e.g., lgb.Booster or lgb.LGBMClassifier).
# - X_test_processed: A pandas DataFrame or numpy array containing the preprocessed test features.
# - test_df: The original test DataFrame, which must contain the 'PassengerId' column.

In [ ]:
def generate_submission_file(
    lgbm_model: lgb.Booster,
    X_test_processed: pd.DataFrame,
    test_df: pd.DataFrame,
    prediction_threshold: float = 0.5,
    submission_filename: str = 'submission.csv'
) -> None:
    """
    Generates a Kaggle-style submission file by performing predictions and formatting the output.

    This function takes a trained LightGBM model, preprocessed test data, and the original
    test DataFrame (for 'PassengerId') to create a CSV file suitable for submission.

    Args:
        lgbm_model: The trained LightGBM model object (e.g., lgb.Booster).
        X_test_processed: The preprocessed test features (e.g., a pandas DataFrame).
        test_df: The original test DataFrame, which must include a 'PassengerId' column.
        prediction_threshold: The threshold to convert predicted probabilities into binary
                              class labels (0 or 1). Defaults to 0.5.
        submission_filename: The name of the CSV file to save the submission to.
                             Defaults to 'submission.csv'.

    Raises:
        ValueError: If the 'PassengerId' column is missing from 'test_df'.
    """
    print("--- Starting Prediction and Submission File Generation ---")

    # 1. Prediction
    print("Predicting survival probabilities on the test dataset...")
    predictions_proba = lgbm_model.predict(X_test_processed)
    print(f"Shape of predicted probabilities array: {predictions_proba.shape}")
    print(f"First 5 predicted probabilities: {predictions_proba[:5]}")

    print(f"Converting probabilities to binary predictions using a threshold of {prediction_threshold}...")
    predictions = (predictions_proba > prediction_threshold).astype(int)
    print(f"Shape of binary predictions array: {predictions.shape}")
    print(f"First 5 binary predictions: {predictions[:5]}")

    # 2. Submission File Generation
    print("Creating submission DataFrame in Kaggle format...")

    if 'PassengerId' not in test_df.columns:
        raise ValueError("The 'test_df' DataFrame must contain a 'PassengerId' column.")

    submission_df = pd.DataFrame({
        'PassengerId': test_df['PassengerId'],
        'Survived': predictions
    })

    print(f"Shape of the generated submission DataFrame: {submission_df.shape}")
    print("First 5 rows of the submission DataFrame:")
    print(submission_df.head())

    print(f"Saving submission DataFrame to '{submission_filename}'...")
    submission_df.to_csv(submission_filename, index=False)

    print(f"\nSuccessfully created '{submission_filename}'. It is ready for Kaggle submission.")
    print("--- Finished Prediction and Submission File Generation ---")